In [4]:
import numpy as np
from numba import jit, prange, njit, vectorize
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline 
import itertools as it
import warnings as w
import os 
import pylab
import math
import time

In [5]:
R=0.75 #cm
H=0.21 #cm

N=500 # With N=1000 it did not converge after 10 hours.
M=200

hr=R/N
hz=H/M

e=4.80324e-10

position_of_electrons=100 #96#48#

N_Vib=np.round(0.4/hr)
N_Vmb=np.round(0.57/hr)

N_Vit=N_Vib
N_Vmt=N_Vmb

x=np.linspace(0,R,N+1)

eps = 1e-2

In [7]:
temp = np.zeros((N+1, N+1))

for i in range(2, N+1):
    for n in range(2, N+1):
        if(i == n):
            temp[i-1, n-1] = (i-1)/(2*(i-1)*(hz/hr+hr/hz))

preCalcIrange = np.arange(2, N+1)

@njit
def GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG):

    hr=R/N
    hz=H/M
    G=initialG
    max_err=1
    err = np.zeros((M+1, N+1, N+1))
    count = 0

    while(max_err > eps):
    
        #a = np.copy(G[1:M, 0:N, 0])
        a = np.copy(G[1:M, 0:N, 0:N])
        print(max_err)

        G[1:position_of_electrons, 0, 0] =(4*(hz/hr)*G[1:position_of_electrons,1,0]+(hr/hz)*G[0:position_of_electrons-1,0,0]+(hr/hz)*G[2:position_of_electrons+1,0,0])/(4*hz/hr+2*hr/hz)

        G[position_of_electrons, 0, 0] = (4 * (hz/hr) * G[position_of_electrons, 1, 0] + (hr/hz) * G[position_of_electrons-1, 0, 0] + (hr/hz) * G[position_of_electrons+1, 0, 0] + 1)/(4 * hz/hr + 2 * hr/hz)

        G[position_of_electrons+1:M, 0, 0] = (4*(hz/hr)*G[position_of_electrons+1:M,1,0]+(hr/hz)*G[position_of_electrons:M-1,0,0]+(hr/hz)*G[position_of_electrons+2:M+1,0,0])/(4*hz/hr+2*hr/hz)

        
        
        G[1:M,1:N,0]=((preCalcIrange - 1/2 - 1)*(hz/hr)*G[1:M,0:N-1,0]+(preCalcIrange + 1/2 - 1)*(hz/hr)*G[1:M,2:N+1,0]+(preCalcIrange - 1)*(hr/hz)*G[0:M-1,1:N,0]+(preCalcIrange - 1)*(hr/hz)*G[2:M+1,1:N,0])/(2*(preCalcIrange - 1)*(hz/hr+hr/hz))
            
        #err[1:M, 0:N, 0] = np.abs(G[1:M, 0:N, 0] - a)

        #a = np.copy(G[1:M, 0:N, 1:N])

        G[1:M,0,1:N]=(4*(hz/hr)*G[1:M,1,1:N]+(hr/hz)*G[0:M-1,0,1:N]+(hr/hz)*G[2:M+1,0,1:N])/(4*hz/hr+2*hr/hz)

        G[1:position_of_electrons,1:N,1:N]=((preCalcIrange - 1/2 - 1)*(hz/hr)*G[1:position_of_electrons,0:N-1,1:N]+(preCalcIrange + 1/2 - 1)*(hz/hr)*G[1:position_of_electrons,2:N+1,1:N]+(preCalcIrange-1)*(hr/hz)*G[0:position_of_electrons-1,1:N,1:N]+(preCalcIrange-1)*(hr/hz)*G[2:position_of_electrons+1,1:N,1:N])/(2*(preCalcIrange-1)*(hz/hr+hr/hz))
        
        G[position_of_electrons,1:N,1:N]=((preCalcIrange - 1/2 - 1)*(hz/hr)*G[position_of_electrons,0:N-1,1:N]+(preCalcIrange + 1/2 - 1)*(hz/hr)*G[position_of_electrons,2:N+1,1:N]+(preCalcIrange-1)*(hr/hz)*G[position_of_electrons-1,1:N,1:N]+(preCalcIrange-1)*(hr/hz)*G[position_of_electrons+1,1:N,1:N])/(2*(preCalcIrange-1)*(hz/hr+hr/hz)) + temp[1:N, 1:N]

        G[position_of_electrons+1:M,1:N,1:N]=((preCalcIrange - 1/2 - 1)*(hz/hr)*G[position_of_electrons+1:M,0:N-1,1:N]+(preCalcIrange + 1/2 - 1)*(hz/hr)*G[position_of_electrons+1:M,2:N+1,1:N]+(preCalcIrange-1)*(hr/hz)*G[position_of_electrons:M-1,1:N,1:N]+(preCalcIrange-1)*(hr/hz)*G[position_of_electrons+2:M+1,1:N,1:N])/(2*(preCalcIrange-1)*(hz/hr+hr/hz))
        
        err[1:M, 0:N, 0:N] = np.abs(G[1:M, 0:N, 0:N] - a)


        count += 1
        
        max_err = np.amax(err)
    
    print("Iterations for convergence: " + str(count))
    return G

In [ ]:
initialG=np.zeros((M+1,N+1,N+1))

In [8]:



# ------------ Green Function  ------------ #

eps=1e-2# error allowed for Green function
initialG=np.zeros((M+1,N+1,N+1))
G=GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)
# 
#GreenFuncName = 'Green_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
#np.save(GreenFuncName, G)
# 
#GreenFuncName= 'Green_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
#initialG = np.load(GreenFuncName)




1.0
0.23489932885906042
0.11626330418276727
0.06837395185312901
0.04014626755382977
0.028910379138330264
0.019516468765734984
0.015521386190844325
0.014381172587966395
0.012555122185287579
0.011531837890283142
0.010256326171145774
Iterations for convergence: 12


## Sparse attempt don't use

In [19]:
import numpy as np
from scipy.sparse import lil_matrix

def GreenSOLUT_sparse(R, H, N, M, position_of_electrons, eps, initialG):
    hr = R / N
    hz = H / M
    G = initialG.copy()  # Make a copy to avoid modifying initialG directly
    max_err = np.inf
    
    # Initialize G as a sparse matrix
    G_sparse = lil_matrix(initialG.reshape(-1, N + 1))  # Use lil_matrix for incremental updates
    
    while max_err > eps:
        max_err = 0.0
        
        # Update G for n = 1
        for j in range(1, M):
            for i in range(0, N):  # Corrected to zero-based indexing
                idx = i + j * (N + 1)
                if idx >= G_sparse.shape[0]:
                    continue  # Skip if index is out of bounds
                a = G_sparse[idx, 0]
                if i == 0:  # Corrected to zero-based indexing
                    if j == position_of_electrons:
                        G_sparse[idx, 0] = (4 * (hz / hr) * G_sparse[idx + 1, 0] + (hr / hz) * G_sparse[idx - (N + 1), 0] + (hr / hz) * G_sparse[idx + (N + 1), 0] + 1) / (4 * hz / hr + 2 * hr / hz)
                    else:
                        G_sparse[idx, 0] = (4 * (hz / hr) * G_sparse[idx + 1, 0] + (hr / hz) * G_sparse[idx - (N + 1), 0] + (hr / hz) * G_sparse[idx + (N + 1), 0]) / (4 * hz / hr + 2 * hr / hz)
                else:
                    G_sparse[idx, 0] = ((i - 1/2) * (hz / hr) * G_sparse[idx - 1, 0] + (i + 1/2) * (hz / hr) * G_sparse[idx + 1, 0] + (i) * (hr / hz) * G_sparse[idx - (N + 1), 0] + (i) * (hr / hz) * G_sparse[idx + (N + 1), 0]) / (2 * (i) * (hz / hr + hr / hz))
                
                err = np.abs(G_sparse[idx, 0] - a)
                if err > max_err:
                    max_err = err
        
        # Update G for n > 1
        for n in range(1, N):
            for j in range(1, M):
                for i in range(0, N):  # Corrected to zero-based indexing
                    idx = i + j * (N + 1) + n * (M + 1) * (N + 1)
                    if idx >= G_sparse.shape[0]:
                        continue  # Skip if index is out of bounds
                    a = G_sparse[idx, 0]
                    if i == 0:  # Corrected to zero-based indexing
                        G_sparse[idx, 0] = (4 * (hz / hr) * G_sparse[idx + 1, 0] + (hr / hz) * G_sparse[idx - (N + 1), 0] + (hr / hz) * G_sparse[idx + (N + 1), 0]) / (4 * hz / hr + 2 * hr / hz)
                    else:
                        if j == position_of_electrons and i == n:
                            G_sparse[idx, 0] = ((i - 1/2) * (hz / hr) * G_sparse[idx - 1, 0] + (i + 1/2) * (hz / hr) * G_sparse[idx + 1, 0] + (i) * (hr / hz) * G_sparse[idx - (N + 1), 0] + (i) * (hr / hz) * G_sparse[idx + (N + 1), 0] + (i - 1)) / (2 * (i) * (hz / hr + hr / hz))
                        else:
                            G_sparse[idx, 0] = ((i - 1/2) * (hz / hr) * G_sparse[idx - 1, 0] + (i + 1/2) * (hz / hr) * G_sparse[idx + 1, 0] + (i) * (hr / hz) * G_sparse[idx - (N + 1), 0] + (i) * (hr / hz) * G_sparse[idx + (N + 1), 0]) / (2 * (i) * (hz / hr + hr / hz))
                    
                    err = np.abs(G_sparse[idx, 0] - a)
                    if err > max_err:
                        max_err = err
        
        print(f'max_err = {max_err}')
        # Include additional visualization or progress updates if needed
        
    # Reshape G_sparse back to original shape
    G = G_sparse.toarray().reshape(M + 1, N + 1, N + 1)
    
    return G



# Example usage
initialG = np.zeros((M+1, N+1, N+1))  # Example initialG, adjust dimensions as needed

# Call the function with sparse matrices
G_sparse = GreenSOLUT_sparse(R, H, N, M, position_of_electrons, eps, initialG)


max_err = 0.14227642276422764
max_err = 0.021704237500221324
max_err = 0.008402467820817189
max_err = 0.004583783224402588
max_err = 0.00295715913911318
max_err = 0.002100443390366191
max_err = 0.001587266583185809
max_err = 0.0012523434537734524
max_err = 0.001019950062103231
max_err = 0.0008511176672532372
max_err = 0.0007239993458211524
max_err = 0.0006255132063717472
max_err = 0.0005474050759183835
max_err = 0.00048424023536280947
max_err = 0.0004323115174908465
max_err = 0.00038901355417625405
max_err = 0.0003524679756733984
max_err = 0.00032129031650598994
max_err = 0.0002944402726821338
max_err = 0.00027112274776391887
max_err = 0.00025072081624788467
max_err = 0.00023274929806793576
max_err = 0.00021682196879510562
max_err = 0.00020262798804965287
max_err = 0.00018991468224613395
max_err = 0.00017847478536706096
max_err = 0.0001681379369149888
max_err = 0.0001587613023286416
max_err = 0.00015022323958507888
max_err = 0.00014242253168252061
max_err = 0.0001352731277930036
max_er

In [15]:
print(np.shape(lil_matrix(initialG.reshape(-1, N + 1))))
print(idx)

(201201, 1001)


NameError: name 'idx' is not defined

In [11]:
eps=1e-5
initialG=G
G=GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)

1.0
9.96767985251279e-05
9.945580299808299e-05
9.929184095390509e-05
9.907215776402967e-05
9.890891015040815e-05
9.869052936964384e-05
9.852799432957804e-05
9.831090611082949e-05
9.814908173504111e-05
9.79332763118429e-05
9.777216064082994e-05
9.755762832698578e-05
9.739721935317358e-05
9.71839505422567e-05
9.702424621203798e-05
9.681223137675277e-05
9.665322959258316e-05
9.644245928458473e-05
9.628415790662037e-05
9.607462275591783e-05
9.591701960409704e-05
9.570871031849837e-05
9.555180317426248e-05
9.534471053895821e-05
9.518849714700017e-05
9.498261202407765e-05
9.482709009402124e-05
9.462240342190953e-05
9.446757062989142e-05
9.42640734229172e-05
9.410992741321067e-05
9.39387924565327e-05
9.37541491475985e-05
9.362002179082174e-05
9.343631314936984e-05
9.330267450277652e-05
9.311991648010276e-05
9.298674474533808e-05
9.280493139260382e-05
9.267222664247066e-05
9.249135204325576e-05
9.23591142911323e-05
9.217917256157393e-05
9.204740176335657e-05
9.18683870523157e-05
9.173708310811

## Old green testing

In [ ]:
eps=1e-5
initialG=G
G=GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)
# 
GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
np.save(GreenFuncName, G)
# 
eps=1e-6
initialG=G
G=GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)
# 
GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
np.save(GreenFuncName, G)
# 
eps=1e-7
initialG=G
G=GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)
# 
GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
np.save(GreenFuncName, G)

eps=1e-8
initialG = G
#initialG=G
G = GreenSOLUTGreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)

GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
np.save(GreenFuncName, G)

eps = 1e-8
GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
G = np.load(GreenFuncName)

eps = 1e-9
initialG = G
G = GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)

GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
np.save(GreenFuncName, G)

eps = 1e-10
initialG = G
G = GreenSOLUT(R,H,N,M,position_of_electrons,eps,initialG)

GreenFuncName = 'OLDGreen_R' + str(R).replace('.','pt') + '_H' + str(H).replace('.','pt') + '_N' + str(N) + '_M' + str(M) + '_posiE' + str(position_of_electrons) + '_epsE' + str(np.log10(eps)) + '.npy'
np.save(GreenFuncName, G)

# here was written GreenSOLUTGreenSOLUT

1.0
0.008506631614301985
0.00786046187904671
0.007160776059126184
0.006648974875680502
0.006107201685970481
0.005706495047705817
0.005428695674017922
0.005167840587046613
0.004984568035532391
0.0047555382648884065
0.004587801996462484
0.00438616182349319
0.0042337013501351495
0.004055512746554635
0.003917317584871655
0.003759199856112172
0.003633993308909622
0.003493089217361356
0.0033795560470174235
0.003253466024565832
0.0031503560526199292
0.003037067562103146
0.002943238058149167
0.0028410607292680135
0.0027557226086017406
0.002677881805048743
0.0026141073209550864
0.0025635261165129486
0.0025036713603727306
0.0024554747773404273
0.002399286780634849
0.002353445109708946
0.0023006741244426254
0.0022571223797788476
0.002207527969638967
0.002166179576272076
0.0021195339899095778
0.002080290079653463
0.002036379891986212
0.001999135598392654
0.001957762259308743
0.001922410949631942
0.0018833906561968594
0.001849826738871574
0.0018129899021461504
0.0017811106480040417
0.00174630113078